In [110]:
import numpy as np
from matplotlib import pyplot as plot
from scipy.optimize import minimize as min
from scipy.signal import lti
import sympy as sp
import math
np.set_printoptions(precision=2)
from numpy import exp
from steady_state_values import steady_state

In [127]:
# path = 'C:/Users/Greg/Desktop/Gregs Workshop/CBT/Project/Code/Simulation and step testing/CBT-project-CSTR-/System simulation/Steptesting and modelling/Code/fit_results.csv'
filename = 'Fit_results.csv'
import csv
with open(filename) as p:
    #reads csv into a list of lists
    my_list = [rec for rec in csv.reader(p, delimiter=',')]

all_params = [[float(i) for i in my_list[j]] for j,lis in enumerate(my_list)]

def round2SignifFigs(vals,n):
    import numpy as np
    np.set_printoptions(precision=2)
    """
    (list, int) -> numpy array
    (numpy array, int) -> numpy array

    In: a list/array of values
    Out: array of values rounded to n significant figures

    Does not accept: inf, nan, complex

    >>> m = [0.0, -1.2366e22, 1.2544444e-15, 0.001222]
    >>> round2SignifFigs(m,2)
    array([  0.00e+00,  -1.24e+22,   1.25e-15,   1.22e-03])
    """
    if np.all(np.isfinite(vals)) and np.all(np.isreal((vals))):
        eset = np.seterr(all='ignore')
        mags = 10.0**np.floor(np.log10(np.abs(vals)))  # omag's
        vals = np.around(vals/mags,n)*mags             # round(val/omag)*omag
        np.seterr(**eset)
        vals[np.where(np.isnan(vals))] = 0.0           # 0.0 -> nan -> 0.0
    else:
        raise IOError('Input must be real and finite')
    return vals

all_params = [round2SignifFigs(i,2) for i in all_params]



In [112]:
def get_xfer(params,type):
    s = sp.Symbol('s')
    e = sp.Symbol('e')
    
    if type == 'FOPTD':
        k,tau,theta = params
        
        return k*(sp.exp(-theta*s))/(tau*s+1)
    elif type == 'SOPTD':
        k,tau,zeta,theta = params
        return k*sp.exp(-theta*s)/(tau**2 *s**2 + 2*zeta*tau*s + 1)
    
    elif type == 'SOZPTD':
        c1,c2,tau1,tau2,theta = params
        return (c1*s + c2)*sp.exp(-theta*s)/((tau1*s + 1)*(tau2*s+1))

In [113]:
Mvs = ['Ps1','Ps2','Ps3']
outputs = ['Cc_measured','T','H']
Dvs = ['Cao','Tbo','F1']

In [114]:
stepped_vars = ['Ps1','Ps2','Ps3','Cao','Tbo','F1']
outputs = ['Cc_measured', 'T', 'H']
names = []
for i, input in enumerate(stepped_vars):
    for j, output in enumerate(outputs):
        names.append(str(input) + str(output))

def get_type(name):                                                                  # based on intuition after seeing curves
    if name == 'F1T' or name == 'Ps3T' or name == 'Ps3Cc_measured':
        fit_type = 'SOPTD'

    elif name == 'Ps2T' or name == 'Ps2Cc_measured' or name == 'F1Cc_measured':
        fit_type = 'SOZPTD'

    else:
        fit_type = 'FOPTD'

    return fit_type

In [115]:
types = [get_type(name) for name in names]
types[15] = 'FOPTD'
funcs = [get_xfer(count,types[i]) for i,count in enumerate(all_params)]
all_funcs = dict(zip(names,funcs))

Now we just need to put the xfer funcs in the correct matrix order. There's definitely a smarter way to do this but I'm just going to hard code it.

In [123]:
# process
ps1cc = all_funcs['Ps1Cc_measured']
ps1t = all_funcs['Ps1T']
ps1h = all_funcs['Ps1H']
ps2cc = all_funcs['Ps2Cc_measured']
ps2t = all_funcs['Ps2T']
ps2h = all_funcs['Ps2H']
ps3cc = all_funcs['Ps3Cc_measured']
ps3t = all_funcs['Ps3T']
ps3h = all_funcs['Ps3H']

Gp_sym = sp.Matrix([[ps1cc,ps2cc,ps3cc],
                [ps1t,ps2t,ps3t],
                [ps1h,ps2h,ps3h]])

# Disturbance
caocc = all_funcs['CaoCc_measured']
caot = all_funcs['CaoT']
caoh = all_funcs['CaoH']
tbocc = all_funcs['TboCc_measured']
tbot = all_funcs['TboT']
tboh = all_funcs['TboH']
f1cc = all_funcs['F1Cc_measured']
f1t = all_funcs['F1T']
f1h = all_funcs['F1H']

Gd_sym = sp.Matrix([[caocc,tbocc,f1cc],
                  [caot,tbot,f1t],
                  [caoh,tboh,f1h]])
 
# hard coded transfer functions (gonna have to do this)

def g11(s):
    return 

def g12(s):
    return 

def g13(s):
    return 

def g21(s):
    return 

def g22(s):
    return 

def g23(s):
    return 

def g31(s):
    return 

def g32(s):
    return 

def g33(s):
    return 

def gd11(s):
    return 

def gd12(s):
    return 

def gd13(s):
    return 

def gd21(s):
    return 

def gd22(s):
    return 

def gd23(s):
    return

def gd31(s):
    return 

def gd32(s):
    return 

def g33(s):
    return 



def Gp(s):
    return np.matrix([[g11(s),g12(s),g13(s)],
                      [g21(s),g22(s),g23(s)],
                      [g31(s),g32(s),g33(s)]])
                     
def Gd(s):
    return np.matrix([[gd11(s),gd12(s),gd13(s)],
                      [gd21(s),gd22(s),gd23(s)],
                      [gd31(s),gd32(s),gd33(s)]])

def get_sym_freq_resp(tf):
    w = sp.Symbol('omega')
    new = tf.subs(s,1j*w)
    return new



In [124]:
sp.init_printing(use_latex='mathjax')
Gp_sym,Gd_sym

⎛⎡         -138.0⋅s                         -187.0⋅s                -135.0⋅s  
⎜⎢-0.0138⋅ℯ           (-1.15⋅s + 0.000181)⋅ℯ              -0.00284⋅ℯ          
⎜⎢──────────────────  ──────────────────────────────  ────────────────────────
⎜⎢   162.0⋅s + 1       (122.0⋅s + 1)⋅(343.0⋅s + 1)             2              
⎜⎢                                                    9428.41⋅s  + 132.4444⋅s 
⎜⎢                                                                            
⎜⎢         -34.5⋅s                        -113.0⋅s                 -16.2⋅s    
⎜⎢ -0.234⋅ℯ            (-18.6⋅s + 0.232)⋅ℯ                 -0.126⋅ℯ           
⎜⎢ ────────────────    ───────────────────────────    ────────────────────────
⎜⎢   198.0⋅s + 1       (116.0⋅s + 1)⋅(348.0⋅s + 1)             2              
⎜⎢                                                    8190.25⋅s  + 123.623⋅s +
⎜⎢                                                                            
⎜⎢         -17.7⋅s                   -43.3⋅s        

In [118]:
def freq_resp(tf_func,w):
    sval = 1j*w
    gep = tf.subs(s,sval)
    new = np.matrix(gep)
    return new

Process Scaling


In [119]:
from Scaling import scaling,umax,dmax,emax
import numpy as np
scale = scaling(umax,dmax,emax)
new = scale.get_scaled_xfer(Gp_sym,Gd_sym)


In [120]:
Gp_sym = new[0]
Gd_sym = new[1]

In [121]:
Gp_sym

⎡                  -138.0⋅s                                          -187.0⋅s 
⎢-3.0110723280011⋅ℯ           218.193646956602⋅(-1.15⋅s + 0.000181)⋅ℯ         
⎢───────────────────────────  ─────────────────────────────────────────────── 
⎢        162.0⋅s + 1                    (122.0⋅s + 1)⋅(343.0⋅s + 1)           
⎢                                                                             
⎢                                                                             
⎢             -34.5⋅s                                        -113.0⋅s         
⎢     -1.872⋅ℯ                        8.0⋅(-18.6⋅s + 0.232)⋅ℯ                 
⎢     ────────────────                ───────────────────────────────         
⎢       198.0⋅s + 1                     (116.0⋅s + 1)⋅(348.0⋅s + 1)           
⎢                                                                             
⎢                                                                             
⎢                   -17.7⋅s                         

In [122]:
Gd_sym

⎡                   -116.0⋅s                       -136.0⋅s                   
⎢  1.5500476679797⋅ℯ            0.477298602717566⋅ℯ          1.05615325647461⋅
⎢  ─────────────────────────    ───────────────────────────  ─────────────────
⎢         191.0⋅s + 1                    73.4⋅s + 1                 170.0⋅s + 
⎢                                                                             
⎢               -42.5⋅s                      -22.5⋅s                        -9
⎢      1.03008⋅ℯ                      0.962⋅ℯ                    0.6937964⋅ℯ  
⎢      ────────────────               ──────────────         ─────────────────
⎢         82.5⋅s + 1                    74.4⋅s + 1                    2       
⎢                                                            4502.41⋅s  + 88.1
⎢                                                                             
⎢                     -3.57⋅s                      -22.5⋅s                    
⎢-0.0278374767110224⋅ℯ           21.5665790967397⋅ℯ 

In [41]:
s = sp.Symbol('s')

In [43]:
k = ps2t*s

NotImplementedError: multivariate partial fraction decomposition

In [61]:
np.ones_like(10)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])